# Installations

In [ ]:
import os
import sys
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from PIL import Image
import gc
import pickle
from tqdm.auto import tqdm
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig
from dotenv import load_dotenv
from torch.utils.data import DataLoader
import shutil

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print("TRANSFORMERS_CACHE:", os.getenv("TRANSFORMERS_CACHE"))

In [ ]:
os.environ.pop("TRANSFORMERS_CACHE", None)

In [ ]:
os.environ["HF_HOME"] = "/storage/coda1/p-dsgt_clef2025/0/kthakrar3/mediqa-magic-v2/.hf_cache"

# where the downloaded model goes

In [ ]:
print("TRANSFORMERS_CACHE:", os.getenv("TRANSFORMERS_CACHE"))
print("HF_HOME:", os.getenv("HF_HOME"))

In [ ]:
print(sys.executable)

print(f"Python version: {sys.version}")
print(f"Python version info: {sys.version_info}")

In [ ]:
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

# Data loading + preprocessing

In [ ]:
train_csv_file = os.path.join("2025_dataset", "train", "final_df.csv")
train_images_dir = os.path.join("2025_dataset", "train", "images_train")

train_df = pd.read_csv(train_csv_file)

train_df['image_ids'] = train_df['image_ids'].apply(eval)
train_df['responses_en'] = train_df['responses_en'].apply(eval)

train_df.head()

In [ ]:
train_df = train_df.head(10)  # Start with 10 samples for quick debugging
print(f"Using {len(train_df)} samples for training")

In [ ]:
train_df = train_df[['encounter_id', 'qid', 'question_en', 'options_en', 'answer_text', 'image_ids']]

print(f"Filtered dataframe shape: {train_df.shape}")
print("Columns:", train_df.columns.tolist())

display("Sample row:", train_df.head(3))

In [ ]:
def process_batch(batch_df, batch_idx, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    batch_data = []
    
    for idx, row in tqdm(batch_df.iterrows(), total=len(batch_df), desc=f"Batch {batch_idx}"):
        try:
            image_paths = [os.path.join(train_images_dir, img_id) for img_id in row['image_ids']]
            
            if not all(os.path.exists(img_path) for img_path in image_paths):
                continue

            valid_images = []
            for img_path in image_paths:
                try:
                    with Image.open(img_path) as img:
                        img.load()
                    valid_images.append(img_path)
                except Exception as e:
                    print(f"Corrupt or unreadable image at {img_path} — {e}")

            if len(valid_images) != len(image_paths):
                continue
            
            options_text = ", ".join([f"{i+1}. {opt}" for i, opt in enumerate(eval(row['options_en']))])
            query_text = f"Question: {row['question_en']} Options: {options_text}"
            
            batch_data.append({
                "encounter_id": row['encounter_id'],
                "qid": row['qid'],
                "query_text": query_text,
                "image_paths": valid_images,
                "answer_text": row['answer_text']
            })
        
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
    
    batch_file = os.path.join(save_dir, f"batch_{batch_idx}.pkl")
    with open(batch_file, 'wb') as f:
        pickle.dump(batch_data, f)
    
    return len(batch_data)

In [ ]:
class MedicalImageDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, processor):
        self.processor = processor
        self.examples = []
        
        for batch_file in sorted(os.listdir(data_dir)):
            if batch_file.startswith("batch_") and batch_file.endswith(".pkl"):
                with open(os.path.join(data_dir, batch_file), 'rb') as f:
                    batch_data = pickle.load(f)
                    self.examples.extend(batch_data)
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, idx):
        example = self.examples[idx]
        
        images = [Image.open(path).convert("RGB") for path in example['image_paths']]
        
        messages = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are an AI assistant answering medical questions based on images."}],
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": example['query_text']},
                    *[{"type": "image", "image": img} for img in images],
                ],
            },
            {
                "role": "assistant",
                "content": [{"type": "text", "text": example['answer_text']}],
            },
        ]
        
        return {"messages": messages}

def preprocess_dataset(df, batch_size=50, save_dir="processed_data"):
    total_processed = 0
    
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        batch_idx = i // batch_size
        
        print(f"Processing batch {batch_idx+1}/{(len(df)-1)//batch_size + 1}")
        processed = process_batch(batch_df, batch_idx, save_dir)
        total_processed += processed
        
        gc.collect()
        
        print(f"Processed {total_processed} examples so far")
    
    return total_processed

In [ ]:
processed_data_dir = "processed_data_debug"

if os.path.exists(processed_data_dir):
    shutil.rmtree(processed_data_dir)

In [ ]:
total_examples = preprocess_dataset(train_df, batch_size=500, save_dir=processed_data_dir)
print(f"Total processed examples: {total_examples}")

In [ ]:
batch_file = os.path.join(processed_data_dir, "batch_0.pkl")
with open(batch_file, 'rb') as f:
    batch_data = pickle.load(f)

print("\nSample of processed data (first example):")
sample_data = batch_data[0]
print(f"Encounter ID: {sample_data['encounter_id']}")
print(f"Question ID: {sample_data['qid']}")
print(f"Query text: {sample_data['query_text']}")
print(f"Number of images: {len(sample_data['image_paths'])}")
print(f"Image paths: {sample_data['image_paths']}")
print(f"Answer text: {sample_data['answer_text']}")

In [ ]:
batch_file = os.path.join(processed_data_dir, "batch_0.pkl")
with open(batch_file, 'rb') as f:
    batch_data = pickle.load(f)

print("\nSample of processed data (first example):")
sample_data = batch_data[1]
print(f"Encounter ID: {sample_data['encounter_id']}")
print(f"Question ID: {sample_data['qid']}")
print(f"Query text: {sample_data['query_text']}")
print(f"Number of images: {len(sample_data['image_paths'])}")
print(f"Image paths: {sample_data['image_paths']}")
print(f"Answer text: {sample_data['answer_text']}")

In [ ]:
batch_file = os.path.join(processed_data_dir, "batch_0.pkl")
with open(batch_file, 'rb') as f:
    batch_data = pickle.load(f)

print("\nSample of processed data (first example):")
sample_data = batch_data[2]
print(f"Encounter ID: {sample_data['encounter_id']}")
print(f"Question ID: {sample_data['qid']}")
print(f"Query text: {sample_data['query_text']}")
print(f"Number of images: {len(sample_data['image_paths'])}")
print(f"Image paths: {sample_data['image_paths']}")
print(f"Answer text: {sample_data['answer_text']}")

In [ ]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
model_id = "google/gemma-3-4b-pt"
processor = AutoProcessor.from_pretrained(model_id, token=hf_token, use_fast=True)

In [ ]:
dataset = MedicalImageDataset(processed_data_dir, processor)
print(f"\nDataset size: {len(dataset)}")

example = dataset[0]
print(f"\nFirst example roles:")
print(f"System role: {example['messages'][0]}")
print(f"User role: {example['messages'][1]}")
print(f"Assistant role: {example['messages'][2]}")

In [ ]:
dataset = MedicalImageDataset(processed_data_dir, processor)
print(f"\nDataset size: {len(dataset)}")

example = dataset[1]
print(f"\nFirst example roles:")
print(f"System role: {example['messages'][0]}")
print(f"User role: {example['messages'][1]}")
print(f"Assistant role: {example['messages'][2]}")

In [ ]:
dataset = MedicalImageDataset(processed_data_dir, processor)
print(f"\nDataset size: {len(dataset)}")

example = dataset[2]
print(f"\nFirst example roles:")
print(f"System role: {example['messages'][0]}")
print(f"User role: {example['messages'][1]}")
print(f"Assistant role: {example['messages'][2]}")

# Load model

In [ ]:
model_id = "google/gemma-3-4b-pt"

if torch.cuda.get_device_capability()[0] < 8:
    raise ValueError("GPU does not support bfloat16. Use a different GPU.")

model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=model_kwargs["torch_dtype"],
    bnb_4bit_quant_storage=model_kwargs["torch_dtype"],
)

model = AutoModelForImageTextToText.from_pretrained(model_id, **model_kwargs, token=hf_token)

processor = AutoProcessor.from_pretrained(model_id, token=hf_token)

# Collate data

In [ ]:
print(f"Current chat template: {processor.tokenizer.chat_template}")

In [ ]:
gemma_chat_template = """{% for message in messages %}
{% if message['role'] == 'user' %}
<start_of_turn>user
{% for content in message['content'] %}
{% if content['type'] == 'text' %}{{ content['text'] }}{% elif content['type'] == 'image' %}<image>{% endif %}
{% endfor %}
<end_of_turn>
{% elif message['role'] == 'assistant' %}
<start_of_turn>model
{% for content in message['content'] %}
{% if content['type'] == 'text' %}{{ content['text'] }}{% endif %}
{% endfor %}
<end_of_turn>
{% elif message['role'] == 'system' %}
<start_of_turn>system
{% for content in message['content'] %}
{% if content['type'] == 'text' %}{{ content['text'] }}{% endif %}
{% endfor %}
<end_of_turn>
{% endif %}
{% endfor %}
{% if add_generation_prompt %}
<start_of_turn>model
{% endif %}
"""

# Set template
processor.tokenizer.chat_template = gemma_chat_template
processor.chat_template = gemma_chat_template

# Ensure <image> is recognized as a token
if "<image>" not in processor.tokenizer.get_vocab():
    processor.tokenizer.add_special_tokens({'additional_special_tokens': ['<image>']})
    model.resize_token_embeddings(len(processor.tokenizer))

# Set boi_token for template rendering
processor.boi_token = "<image>"
processor.tokenizer.special_tokens_map['boi_token'] = "<image>"

In [ ]:
print(f"Current chat template: {processor.tokenizer.chat_template}")

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
#     target_modules="all-linear",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head", "embed_tokens"],
)

In [ ]:
def process_vision_info(messages):
    """
    Extracts images from a structured messages list.
    Returns a list of PIL Image objects in RGB format.
    """
    image_inputs = []
    # print(f"Processing messages: {messages}")
    
    for msg in messages:
        content = msg.get("content", [])
        # print(f"Message content: {content}")
        
        if not isinstance(content, list):
            content = [content]
        
        for element in content:
            # print(f"Checking element: {type(element)}")
            if isinstance(element, dict) and (
                "image" in element or element.get("type") == "image"
            ):
                # print("Found image element!")
                if "image" in element:
                    image = element["image"]
                else:
                    image = element
                
                if hasattr(image, 'convert'):
                    image = image.convert("RGB")
                    image_inputs.append(image)
                    # print(f"Added image: {image.size}")
                else:
                    # print(f"Element is not a PIL image: {type(image)}")
                    pass
    
    # print(f"Total images found: {len(image_inputs)}")
    return image_inputs

In [ ]:
def create_dummy_image():
    """Create a small black image as a placeholder."""
    return Image.new('RGB', (224, 224), color='black')

In [ ]:
# Ensure <image> is properly registered as a special token
if "<image>" not in processor.tokenizer.get_vocab():
    # Add the token to the vocabulary
    processor.tokenizer.add_special_tokens({'additional_special_tokens': ['<image>']})
    # VERY IMPORTANT: Resize the model's embeddings to match the new vocabulary size
    model.resize_token_embeddings(len(processor.tokenizer))

# Check what token ID is assigned to <image>
image_token_id = processor.tokenizer.convert_tokens_to_ids("<image>")
print(f"<image> token ID: {image_token_id}")

In [ ]:
def collate_fn(examples):
    texts = []
    images_per_example = []
    
    for example in examples:
        image_inputs = process_vision_info(example["messages"])
        if not image_inputs:
            print(f"Using dummy image — Example roles: {[m['role'] for m in example['messages']]}")
            image_inputs = [create_dummy_image()]
        
        # Apply chat template with hardcoded <image> token
        text = processor.apply_chat_template(
            example["messages"], add_generation_prompt=False, tokenize=False
        )
        
        # Count image tokens explicitly
        num_image_tokens = text.count("<image>")
        
        # Ensure number of images matches tokens
        if len(image_inputs) < num_image_tokens:
            needed_dummies = num_image_tokens - len(image_inputs)
            image_inputs += [create_dummy_image()] * needed_dummies
        elif len(image_inputs) > num_image_tokens:
            # Never truncate to zero
            image_inputs = image_inputs[:max(1, num_image_tokens)]
        
        texts.append(text.strip())
        images_per_example.append(image_inputs)
    
    # Use the processor directly as in the documentation
    batch = processor(
        text=texts, 
        images=images_per_example,  # Pass as list of lists
        return_tensors="pt", 
        padding=True
    )
    
    # Create labels for training
    labels = batch["input_ids"].clone()
    
    # Get image token ID the same way the model does
    image_token_id = processor.tokenizer.convert_tokens_to_ids("<image>")
    
    # Mask tokens that shouldn't contribute to the loss
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    labels[labels == 262144] = -100
    
    batch["labels"] = labels
    return batch

In [ ]:
dataset = MedicalImageDataset(processed_data_dir, processor)
print(f"Dataset size: {len(dataset)}")

if len(dataset) == 0:
    print("ERROR: Dataset is empty! Check data loading process.")
    pass
else:
    sample_size = min(3, len(dataset))
    print(f"Sampling {sample_size} examples from dataset")
    
    sample_examples = [dataset[i] for i in range(sample_size)]
    
    print(f"Sample size: {len(sample_examples)}")
    print("First example keys:", list(sample_examples[0].keys()))
    
    batch = collate_fn(sample_examples)
    print("Collated batch contains:", list(batch.keys()))
    print(f"Input_ids shape: {batch['input_ids'].shape}")
    print(f"Labels shape: {batch['labels'].shape}")

In [ ]:
dataloader = DataLoader(
    dataset,
    batch_size=8,  # Adjust based on GPU memory
    shuffle=True,
    collate_fn=collate_fn
)

total_examples = 0
for batch in dataloader:
    # Process each batch. Note: in training, pass this to model.forward()
    batch_size = len(batch["input_ids"])
    total_examples += batch_size
    print(f"Processed batch with {batch_size} examples")

print(f"Processed all {total_examples} examples")

In [ ]:
def debug_template_application(examples):
    print("\n=== DEBUGGING TEMPLATE APPLICATION ===")
    for i, example in enumerate(examples):
        print(f"\nExample {i}:")
        
        # First, check message structure
        print("Message structure:")
        for j, msg in enumerate(example["messages"]):
            role = msg.get("role", "unknown")
            print(f"  Message {j} role: {role}")
            
            content = msg.get("content", [])
            if not isinstance(content, list):
                content = [content]
                
            print(f"  Content types: {[c.get('type') if isinstance(c, dict) else type(c).__name__ for c in content]}")
            
            # Check specifically for image content
            image_count = sum(1 for c in content if isinstance(c, dict) and c.get('type') == 'image')
            print(f"  Image content count: {image_count}")
        
        # Next, check template application
        text = processor.apply_chat_template(
            example["messages"], add_generation_prompt=False, tokenize=False
        )
        
        # Count image tokens in generated text
        img_token_count = text.count("<image>")
        print(f"  Image tokens in text: {img_token_count}")
        
        # Print text snippet to see if tokens appear
        print(f"  Text preview: {text[:200]}...")
        
        # Check if there's a mismatch
        if img_token_count == 0:
            print("  WARNING: No image tokens found in text!")
            
            # Examine raw content in more detail
            print("  Detailed content examination:")
            for j, msg in enumerate(example["messages"]):
                if msg.get("role") == "user":
                    content = msg.get("content", [])
                    if not isinstance(content, list):
                        content = [content]
                    
                    for k, item in enumerate(content):
                        if isinstance(item, dict):
                            print(f"    Item {k}: type={item.get('type')}, keys={list(item.keys())}")
                            if item.get('type') == 'image':
                                print(f"    Found image content item")
                            else:
                                print(f"    Item has type={item.get('type')}, not 'image'")
                                
debug_template_application(sample_examples)

In [ ]:
args = SFTConfig(
    output_dir="gemma-product-description",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=True,
    report_to="tensorboard",
    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_text_field="",
    dataset_kwargs={"skip_prepare_dataset": True},
    remove_unused_columns=False,  # Critical for custom datasets
    label_names=["labels"],  # Explicitly setting label_names
)

# args.remove_unused_columns = False # Not needed in our case but leaving it in in case the loaded data changes

In [ ]:
# # Test with a small batch
# test_indices = list(range(min(3, len(dataset))))
# test_batch = [dataset[i] for i in test_indices]

# Now initialize your actual trainer with the regular collate_fn
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=processor,
    data_collator=collate_fn,
)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Save the model
trainer.save_model()
print("Training complete and model saved!")

In [ ]:
# Before you can test your model, make sure to free the memory.

# free the memory again
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
from peft import PeftModel

# Load Model base model
model = AutoModelForImageTextToText.from_pretrained(model_id, low_cpu_mem_usage=True)

# Merge LoRA and base model
peft_model = PeftModel.from_pretrained(model, args.output_dir)
merged_model = peft_model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True, max_shard_size="2GB")

processor = AutoProcessor.from_pretrained(args.output_dir)
processor.save_pretrained("merged_model")